In [31]:
import numpy as np
import json
import pickle
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [32]:
## Loading word to index mapping
with open('./resources/word2idx.pkl','rb') as f:
    word2idx=pickle.load(f)

## Loading index to word mapping
with open('./resources/idx2word.pkl','rb') as f:
    idx2word=pickle.load(f)

## Loading ResNet50 testing image features     
with open("./resources/encoded_test_images.pkl","rb") as f:    
    encoded_test_images = pickle.load(f)

In [39]:
maxlen = 39
model = load_model('./models_weights/model.h5')

def predict_caption(photo):
    input_text = '<s>'
    for i in range(maxlen):
        sequence = [word2idx[word] for word in input_text.split() if word in word2idx]
        sequence = pad_sequences([sequence],maxlen=maxlen,padding='post')
        
        ypred = model.predict([photo,sequence])
        ypred = ypred.argmax()
        word = idx2word[ypred]
        
        input_text +=' ' + word
        
        if word == '<e>':
            break
            
    final_caption=input_text.split()[1:-1]
    final_caption=' '.join(final_caption)
    
    return final_caption

In [41]:
## pick and generating captions for random images from test set
all_image_names = list(encoded_test_images.keys())
for i in range(15):
    idx=np.random.randint(0,1000)
    image_name = all_image_names[idx]
    photo = encoded_test_images[image_name].reshape(1,2048)
    
    caption = predict_caption(photo)
    print(caption)
    i=plt.imread('./flickr8k/Data/Images/'+image_name)
    plt.imshow(i)
    plt.show()